In [4]:
import cv2
import numpy as np 

In [5]:
def nothing(x):
    # any operation
    pass

In [6]:
def rescale_frame(frame_input, percent=30):
    width = int(frame_input.shape[1] * percent / 100)
    height = int(frame_input.shape[0] * percent / 100)
    dim = (width, height)
    return cv2.resize(frame_input, dim, interpolation=cv2.INTER_AREA)

In [7]:
cap = cv2.VideoCapture(1)

cv2.namedWindow("Trackbars")
cv2.createTrackbar("thre1", "Trackbars", 120, 255, nothing)
cv2.createTrackbar("thre2", "Trackbars", 60, 255, nothing)
cv2.createTrackbar("thre3", "Trackbars", 20, 120, nothing)

font = cv2.FONT_HERSHEY_SIMPLEX

while True:
    _, frame = cap.read()
    if frame is None:
        break

    frame = rescale_frame(frame)

    thre1 = cv2.getTrackbarPos("thre1", "Trackbars")
    thre2 = cv2.getTrackbarPos("thre2", "Trackbars")

    thre3 = cv2.getTrackbarPos("thre3", "Trackbars")

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)
    dilated_img = cv2.dilate(blurred_frame, np.ones((7, 7), np.uint8))
    bg_img = cv2.medianBlur(dilated_img, 21)
    diff_img = cv2.absdiff(gray_frame, bg_img)
    norm_img = cv2.normalize(diff_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    canny_frame = cv2.Canny(norm_img, thre1, thre2)
    canny_frame = cv2.dilate(canny_frame, kernel=np.ones((3, 3), np.uint8), iterations=1)

    mask1 = np.ones(frame.shape, dtype=np.uint8)
    contours1, _ = cv2.findContours(canny_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt1 in contours1:
        x, y, w, h = cv2.boundingRect(cnt1)
        # # Create a mask from a boundingRect in openCV
        if w>thre3 and h>thre3 and w<2.1*thre3 and h<2.1*thre3:
            cv2.rectangle(mask1, (x, y), (x + w, y + h), (255, 255, 255), -1)
            mask1 = cv2.erode(mask1, kernel=np.ones((5, 5), np.uint8), iterations=2)
            mask1 = cv2.dilate(mask1, kernel=np.ones((5, 5), np.uint8), iterations=1)

    cv2.imshow("Frame", frame)
    cv2.imshow("Edge", canny_frame)
    cv2.imshow("Mask1", mask1)
    if cv2.waitKey(30) == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()